In [13]:
!pip3 install numpy
!pip3 install pandas
!pip3 install torch
!pip3 install scikit-learn
!pip3 install tensorflow
!pip3 install setuptools

# Data Processing

In [14]:
import json
from datetime import datetime

def merge_json_files(performance_file, record_file, output_file):
    with open(performance_file, 'r') as pf:
        performance_data = json.load(pf)

    with open(record_file, 'r') as rf:
        record_data = json.load(rf)
    
    record_data = [
        {**record, "timestamp": datetime.strptime(record["timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ")}
        for record in record_data
    ]
    
    merged_data = []

    for perf_sample in performance_data:
        perf_timestamp = datetime.strptime(perf_sample["timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ")

        closest_record = None
        min_diff = float('inf')

        for record in record_data:
            diff = abs((perf_timestamp - record["timestamp"]).total_seconds())
            if diff < min_diff:
                min_diff = diff
                closest_record = record

        if closest_record:
            merged_sample = {
                "timestamp": perf_sample["timestamp"],
                "requestPerSec": perf_sample["requestPerSec"],
                "bytesPerSec": perf_sample["bytesPerSec"],
                "bufferSize": perf_sample["bufferSize"],
                "cpuUsage": closest_record["cpuUsage"],
                "ramUsage": closest_record["ramUsage"]
            }
            merged_data.append(merged_sample)

    with open(output_file, 'w') as output:
        json.dump(merged_data, output, indent=4)

merge_json_files('performance.json', 'record.json', 'training_data.json')


## Resource Optimization Model